# Import and save to Gsheet


In [21]:
# Notebook config, enviroment and logging

import os
import sys

import logging
from dotenv import load_dotenv

import pakkenellik.config as conf
from pakkenellik.log import logger

# Add module path and load config
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Autoreload extension
if "autoreload" not in get_ipython().extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Load the .env file into local env
load_dotenv()

# Create the config
config = conf.Config(module_path)

# Enable logging and crank up log level to DEBUG.
# This is particularly useful when developing code in your project module and using it from a notebook.
logger.setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
## Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

In [23]:
## Specific imports
from pakkenellik.google.sheets import (
    get_worksheet_as_dataframe,
    open_or_create_spreadsheet,
    save_dataframe_to_worksheet,
    format_worksheet,
)

## Read from spreadsheet

In [9]:
# Demo, uses historical football matches from the second tier in Norway
spreadsheet_key = "1ar2A28vr8iRIidzgkH16Z7RExDU2W_faYdr7bRpQIA8"
worksheet_name = "historical_obos_games"

df = get_worksheet_as_dataframe(spreadsheet_key, worksheet_name=worksheet_name)

In [10]:
df.head()

,date,year,month,day,_round,home_team,away_team,home_goals_half_time,away_goals_half_time,home_goals_full_time,away_goals_full_time,result,team,opponent_team,home_away,season_game,points,team_goals,team_conceded,team_goals_half_time,team_conceded_half_time,non_win,non_loose,turn_around_win,turn_around_loose,...,season_conceded_total_reverse,season_home_game_reverse,season_home_points_total_reverse,season_home_points_avg_reverse,season_home_points_avg_after,season_away_game_reverse,season_away_points_total_reverse,season_away_points_avg_reverse,season_away_points_avg_after,season_round_points_total,season_round_points_avg,season_round_scored_total,season_round_conceded_total,season_final_rank,season_final_rule,season_final_games,season_final_points,season_final_wins,season_final_draws,season_final_losses,season_final_goals_for,season_final_goals_against,season_final_avg,Unnamed: 69,Unnamed: 70
0,1997-04-20 16:00:00,1997.0,4.0,7.0,1.0,Aalesund,Hødd,4.0,2.0,4.0,2.0,1,Aalesund,Hødd,home,1.0,3.0,4.0,2.0,4.0,2.0,0.0,1.0,NaN,0.0,...,36.0,13.0,21.0,1.615385,1.500000,NaN,NaN,NaN,NaN,3.0,3.00,4.0,2.0,10.0,NaN,26.0,35.0,11.0,2.0,13.0,36.0,36.0,1.346154,NaN,NaN
1,1997-04-27 16:00:00,1997.0,4.0,7.0,2.0,Eik-Tønsberg,Aalesund,3.0,1.0,3.0,1.0,1,Aalesund,Eik-Tønsberg,away,2.0,0.0,1.0,3.0,1.0,3.0,1.0,0.0,0.0,NaN,...,34.0,NaN,NaN,NaN,NaN,13.0,14.0,1.076923,1.166667,3.0,1.50,5.0,5.0,10.0,NaN,26.0,35.0,11.0,2.0,13.0,36.0,36.0,1.346154,NaN,NaN
2,1997-05-04 16:00:00,1997.0,5.0,7.0,3.0,Aalesund,Start,1.0,3.0,1.0,3.0,2,Aalesund,Start,home,3.0,0.0,1.0,3.0,1.0,3.0,1.0,0.0,0.0,NaN,...,31.0,12.0,18.0,1.500000,1.636364,NaN,NaN,NaN,NaN,3.0,1.00,6.0,8.0,10.0,NaN,26.0,35.0,11.0,2.0,13.0,36.0,36.0,1.346154,NaN,NaN
3,1997-05-08 16:00:00,1997.0,5.0,4.0,4.0,Moss,Aalesund,1.0,0.0,1.0,0.0,1,Aalesund,Moss,away,4.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,...,28.0,NaN,NaN,NaN,NaN,12.0,14.0,1.166667,1.272727,3.0,0.75,6.0,9.0,10.0,NaN,26.0,35.0,11.0,2.0,13.0,36.0,36.0,1.346154,NaN,NaN
4,1997-05-16 16:00:00,1997.0,5.0,5.0,5.0,Aalesund,Bryne,0.0,2.0,0.0,2.0,2,Aalesund,Bryne,home,5.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,NaN,...,27.0,11.0,18.0,1.636364,1.800000,NaN,NaN,NaN,NaN,3.0,0.60,6.0,11.0,10.0,NaN,26.0,35.0,11.0,2.0,13.0,36.0,36.0,1.346154,NaN,NaN


## Write to Spreadsheet

In [34]:
new_worksheet = save_dataframe_to_worksheet(
    df,
    "Obos 1997 – 2021",
    "1Ks2h58ja7DmiovfTDEHPB-RNt1IQrf_Q",
    "games",
    include_index=False,
)

In [32]:
df.index

RangeIndex(start=0, stop=11478, step=1)

In [35]:
# Basic formatting.
# See https://github.com/robin900/gspread-formatting for tips on formatting

# Example formatting
text_cols = ["home_team", "away_team", "opponent_team", "home_away", "result"]
bold_text_cols = ["team"]
date_cols = ["date"]
int_cols = [
    "year",
    "month",
    "day",
    "_round",
    "season_game",
    "points",
    "team_goals",
    "team_conceded",
    "team_goals_half_time",
    "team_conceded_half_time",
]
float_cols = [
    "season_points_avg",
    "season_home_points_avg",
    "season_home_points_avg_before",
    "season_final_avg, " "season_away_points_avg",
    "season_away_points_avg_before",
]

format_worksheet(
    new_worksheet,
    df,
    text_columns=text_cols,
    bold_text_columns=bold_text_cols,
    date_columns=date_cols,
    int_columns=int_cols,
    float_columns=float_cols,
    frozen_columns=2,
)